In [1]:
!pip install -q torch-geometric

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np

In [3]:
import pandas as pd
import networkx as nx
import numpy as np
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [4]:

# Load the node_data.csv file and edgelist.csv
node_list = pd.read_csv('/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv')
edgelist = pd.read_csv('/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv')

In [5]:
# node2vec = Node2Vec(, dimensions=64, walk_length=20, num_walks=10)
# model = node2vec.fit(window=10, min_count=1)
# node2vec_embeddings = torch.tensor([model.wv[str(node)] for node in node_list['id']])

In [6]:
# Create a networkx graph from the edge list

G = nx.from_pandas_edgelist(edgelist, source='source', target='target')
G.add_nodes_from(node_list['id'])

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [8]:
# node_colors = []
# for _, node_data in node_list.iterrows():
#     if node_data['branch'] == 'CSE':
#         node_colors.append('blue')
#     elif node_data['branch'] == 'ECE':
#         node_colors.append('green')
#     elif node_data['branch'] == 'EEE':
#         node_colors.append('red')
#     else:
#          node_colors.append('pink')
        

# # Plot the graph before building the model
# plt.figure(figsize=(10, 7))
# nx.draw(G, node_color=node_colors, with_labels=False)
# plt.title('Graph Visualization (Before Building GCN Model)')
# plt.show()

In [9]:
# Extract node features from features.pt file
node_features = torch.load('/kaggle/input/ue20cs344-nam-assignment-2/features.pt')

In [10]:
node_id_map = {node_id: i for i, node_id in enumerate(node_list['id'])}

# Create edge_index tensor from the edgelist
edge_index = torch.tensor([[node_id_map[source], node_id_map[target]] 
                           for source, target in zip(edgelist['source'], edgelist['target'])], 
                          dtype=torch.long).t().contiguous()

In [11]:
# Create train_mask and test_mask tensors from node_data.csv
train_mask = torch.tensor(node_list['train_mask'].values, dtype=torch.bool)
test_mask = torch.tensor(node_list['test_mask'].values, dtype=torch.bool)

# # Create label tensor from node_data.csv
# labels = torch.tensor(node_data_df['branch'].values)



# Create a mapping of branch labels to numerical values
label_map = {label: i for i, label in enumerate(node_list['branch'].unique())}


# Replace branch labels with their corresponding numerical values
node_list['branch'] = node_list['branch'].map(label_map)

# print(label_map["ECE"])
# print(label_map["EEE"])
# print(label_map["CSE"])


# Convert labels to tensor object
labels = torch.tensor(node_list['branch'].values, dtype=torch.long)

# Create a PyG Data object
data = Data(x=node_features, edge_index=edge_index, train_mask=train_mask, test_mask=test_mask, y=labels)

In [12]:
# combined_features = torch.cat([node_features, node2vec_embeddings], dim=1)

# **GCN Model**

In [13]:
# Define the GCN model
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, 3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [14]:
# Initialize the GCN model
input_dim = node_features.shape[1]
hidden_dim = 128
output_dim = 3 # Assuming binary classification problem with two branches
model = GCN(input_dim, hidden_dim, output_dim)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.075)

In [15]:
# Training loop
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(data)
    loss = criterion(logits[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Print training loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the GCN model
model.eval()
with torch.no_grad():
    # Get predictions for the test set
    test_logits = model(data)[data.test_mask]
    _, test_preds = test_logits.max(dim=1)
    test_preds = test_preds.cpu().numpy()

Epoch [1/200], Loss: 1.0979
Epoch [2/200], Loss: 1.0745
Epoch [3/200], Loss: 1.0070
Epoch [4/200], Loss: 0.9294
Epoch [5/200], Loss: 0.8830
Epoch [6/200], Loss: 0.8109
Epoch [7/200], Loss: 0.6896
Epoch [8/200], Loss: 0.6483
Epoch [9/200], Loss: 0.5786
Epoch [10/200], Loss: 0.5066
Epoch [11/200], Loss: 0.4744
Epoch [12/200], Loss: 0.4439
Epoch [13/200], Loss: 0.4384
Epoch [14/200], Loss: 0.4065
Epoch [15/200], Loss: 0.4215
Epoch [16/200], Loss: 0.4173
Epoch [17/200], Loss: 0.4067
Epoch [18/200], Loss: 0.4054
Epoch [19/200], Loss: 0.4033
Epoch [20/200], Loss: 0.4004
Epoch [21/200], Loss: 0.3843
Epoch [22/200], Loss: 0.3893
Epoch [23/200], Loss: 0.3824
Epoch [24/200], Loss: 0.3719
Epoch [25/200], Loss: 0.3685
Epoch [26/200], Loss: 0.3638
Epoch [27/200], Loss: 0.3591
Epoch [28/200], Loss: 0.3497
Epoch [29/200], Loss: 0.3514
Epoch [30/200], Loss: 0.3456
Epoch [31/200], Loss: 0.3407
Epoch [32/200], Loss: 0.3402
Epoch [33/200], Loss: 0.3371
Epoch [34/200], Loss: 0.3338
Epoch [35/200], Loss: 0

In [16]:
print(type(test_preds))

# for i in range(len(test_preds)):
#     if(test_preds[i]==0):
#         test_preds[i] = "ECE"
#     elif(test_preds[i]==1):
#         test_preds[i] = "CSE"
#     elif(test_preds[i]==2):
#         test_preds[i] = "EEE"

label_map_inv = {0: 'ECE', 1: 'CSE', 2: 'EEE'}

# Convert numerical labels to string labels
# test_preds = test_preds.astype(str).map(label_map_inv)

test_preds = np.vectorize(label_map_inv.get)(test_preds)


# Create submission.csv with predicted branches for test set
node_ids = node_list['id'].values
submission_df = pd.DataFrame({'id': node_ids[data.test_mask], 'branch': test_preds})
submission_df.to_csv('submission.csv', index=False)

<class 'numpy.ndarray'>


# **GAT Model**

In [17]:
# Define the GAT model
class GAT(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_heads, dropout):
        super(GAT, self).__init__()
        self.conv1 = gnn.GATConv(input_dim, hidden_dim, heads=num_heads, dropout=dropout)
        self.conv2 = gnn.GATConv(hidden_dim * num_heads, output_dim, concat=False, dropout=dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x



In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric.nn as gnn
from torch_geometric.data import Data

In [19]:
# Initialize the GAT model
input_dim = node_features.shape[1]
hidden_dim = 64
output_dim = 3 
num_heads = 4 
dropout = 0.5 
model = GAT(input_dim, hidden_dim, output_dim, num_heads, dropout)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.025)



In [20]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    logits = model(data)
    loss = criterion(logits[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # Print training loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/100], Loss: 1.0987
Epoch [2/100], Loss: 1.0235
Epoch [3/100], Loss: 0.9873
Epoch [4/100], Loss: 0.9131
Epoch [5/100], Loss: 0.8596
Epoch [6/100], Loss: 0.7998
Epoch [7/100], Loss: 0.7399
Epoch [8/100], Loss: 0.6926
Epoch [9/100], Loss: 0.6567
Epoch [10/100], Loss: 0.6309
Epoch [11/100], Loss: 0.6222
Epoch [12/100], Loss: 0.6154
Epoch [13/100], Loss: 0.6048
Epoch [14/100], Loss: 0.6147
Epoch [15/100], Loss: 0.5866
Epoch [16/100], Loss: 0.5947
Epoch [17/100], Loss: 0.5872
Epoch [18/100], Loss: 0.5775
Epoch [19/100], Loss: 0.5665
Epoch [20/100], Loss: 0.5615
Epoch [21/100], Loss: 0.5446
Epoch [22/100], Loss: 0.5463
Epoch [23/100], Loss: 0.5422
Epoch [24/100], Loss: 0.5310
Epoch [25/100], Loss: 0.5350
Epoch [26/100], Loss: 0.5221
Epoch [27/100], Loss: 0.5191
Epoch [28/100], Loss: 0.5183
Epoch [29/100], Loss: 0.5212
Epoch [30/100], Loss: 0.5100
Epoch [31/100], Loss: 0.5114
Epoch [32/100], Loss: 0.5080
Epoch [33/100], Loss: 0.5039
Epoch [34/100], Loss: 0.4932
Epoch [35/100], Loss: 0

In [21]:
# Evaluate the GAT model
model.eval()
with torch.no_grad():
    # Get predictions for the test set
    test_logits = model(data)[data.test_mask]
    _, test_preds = test_logits.max(dim=1)
    test_preds = test_preds.cpu().numpy()

# Create a mapping of branch labels to numerical values
label_map = {i: label for i, label in enumerate(node_list['branch'].unique())}

# Convert numerical labels to string labels
test_preds = np.vectorize(label_map.get)(test_preds)

# Create submission.csv with predicted branches for test set
node_ids = node_list['id'].values
submission_df = pd.DataFrame({'id': node_ids[data.test_mask], 'branch': test_preds})
# submission_df.to_csv('submission1.csv', index=False)

In [22]:
# import matplotlib.pyplot as plt
# from sklearn.manifold import TSNE

# # Extract node embeddings from the GCN model
# with torch.no_grad():
#     model.eval()
#     embeddings = model(data).detach().numpy()

# # Apply t-SNE for dimensionality reduction
# tsne = TSNE(n_components=2)
# embeddings_tsne = tsne.fit_transform(embeddings)

# # Plot node embeddings before training
# plt.figure(figsize=(10, 5))
# plt.subplot(1, 2, 1)
# plt.scatter(embeddings_tsne[:, 0], embeddings_tsne[:, 1], c=labels.numpy(), cmap='viridis')
# plt.title('Node Embeddings before Training')
# plt.colorbar()